In [146]:
import numpy as np
import pandas as pd
from sklearn import linear_model
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

In [186]:
numeric_columns = df.select_dtypes(['int64', 'float64']).columns
np.abs(df[numeric_columns].iloc[:,1:].corr().loc[:,"saleprice"]).sort_values(ascending=False)


saleprice        1.000000
overallqual      0.790982
grlivarea        0.708624
garagecars       0.640409
garagearea       0.623431
totalbsmtsf      0.613581
firstflrsf       0.605852
fullbath         0.560664
totrmsabvgrd     0.533723
yearbuilt        0.522897
yearremodadd     0.507101
garageyrblt      0.486362
masvnrarea       0.477493
fireplaces       0.466929
bsmtfinsf1       0.386420
foundation       0.382479
lotfrontage      0.351799
wooddecksf       0.324413
secondflrsf      0.319334
openporchsf      0.315856
halfbath         0.284108
lotarea          0.263843
bsmtfullbath     0.227122
bsmtunfsf        0.214479
bedroomabvgr     0.168213
housestyle       0.167173
kitchenabvgr     0.135907
enclosedporch    0.128578
extercond        0.117303
screenporch      0.111447
functional       0.107619
poolarea         0.092404
mssubclass       0.084284
overallcond      0.077856
mosold           0.046432
threessnporch    0.044584
yrsold           0.028923
lowqualfinsf     0.025606
miscval     

In [147]:
df.housestyle = df.housestyle.apply(str.strip)

for i in range (0, len(df.housestyle)):
        if df.housestyle[i] == '1.5Fin':
            df.housestyle[i] = 0
        elif df.housestyle[i] == '1.5Unf':
            df.housestyle[i] = 1
        elif df.housestyle[i] == '1Story':
            df.housestyle[i] = 2
        elif df.housestyle[i] == '2.5Fin':
            df.housestyle[i] = 3
        elif df.housestyle[i] == '2.5Unf':
            df.housestyle[i] = 4
        elif df.housestyle[i] == '2Story':
            df.housestyle[i] = 5
        elif df.housestyle[i] == 'SFoyer':
            df.housestyle[i] = 6
        elif df.housestyle[i] == 'SLv1':
            df.housestyle[i] = 7
        else:
            df.housestyle[i] = 8

In [148]:
df["is_street_paved"] = pd.get_dummies(df.street, drop_first=True)

df.extercond = df.extercond.apply(str.strip)
np.unique(df.extercond)

for i in range (0, len(df.extercond)):
    if df.extercond[i] == 'Ex':
        df.extercond[i] = 0
    elif df.extercond[i] == 'Fa':
        df.extercond[i] = 1
    elif df.extercond[i] == 'Gd':
        df.extercond[i] = 2
    elif df.extercond[i] == 'Po':
        df.extercond[i] = 3
    else:
        df.extercond[i] = 4


In [149]:
df.foundation = df.foundation.apply(str.strip)
np.unique(df.foundation)

for i in range (0, len(df.foundation)):
    if df.foundation[i] == 'BrkTil':
        df.foundation[i] = 0
    elif df.foundation[i] == 'CBlock':
        df.foundation[i] = 1
    elif df.foundation[i] == 'PConc':
        df.foundation[i] = 2
    elif df.foundation[i] == 'Slab':
        df.foundation[i] = 3
    elif df.foundation[i] == 'Stone':
        df.foundation[i] = 4
    else:
        df.foundation[i] = 5

In [150]:
df.functional = df.functional.apply(str.strip)
np.unique(df.functional)

for i in range (0, len(df.functional)):
    if df.functional[i] == 'Typ':
        df.functional[i] = 0
    elif df.functional[i] == 'Min1':
        df.functional[i] = 1
    elif df.functional[i] == 'Min2':
        df.functional[i] = 2
    elif df.functional[i] == 'Mod':
        df.functional[i] = 3
    elif df.functional[i] == 'Maj1':
        df.functional[i] = 4
    elif df.functional[i] == 'Maj2':
        df.functional[i] = 5
    elif df.functional[i] == 'Sev':
        df.functional[i] = 6
    else:
        df.functional[i] = 7
        

In [188]:
X = df[['housestyle', 'is_street_paved', 'overallqual', 'grlivarea', 'garagecars', 'garagearea', 'totalbsmtsf']]
df.housestyle = df['housestyle'].astype(str).astype(int)
df.extercond = df['extercond'].astype(str).astype(int)
df.foundation = df['foundation'].astype(str).astype(int)
df.functional = df['functional'].astype(str).astype(int)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:              saleprice   R-squared (uncentered):                   0.960
Model:                            OLS   Adj. R-squared (uncentered):              0.960
Method:                 Least Squares   F-statistic:                              5014.
Date:                Sat, 22 Feb 2020   Prob (F-statistic):                        0.00
Time:                        22:54:09   Log-Likelihood:                         -17521.
No. Observations:                1460   AIC:                                  3.506e+04
Df Residuals:                    1453   BIC:                                  3.509e+04
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
housestyle         14.3052    570.403      0.025      0.980   -1104.597    1133.207
is_street_paved -9.249e+04   4672.236    -19.796      0.000   -1.02e+05   -8.33e+04
overallqual      2.273e+04   1097.938     20.698      0.000    2.06e+04    2.49e+04
grlivarea          45.4008      2.588     17.546      0.000      40.325      50.476
garagecars       1.506e+04   3071.963      4.904      0.000    9039.039    2.11e+04
garagearea         13.2550     10.638      1.246      0.213      -7.612      34.122
totalbsmtsf        31.3072      3.156      9.920      0.000      25.117      37.498
==============================================================================
Omnibus:                      411.657   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            34761.528
Skew:                          -0.224   Prob(JB):                         0.00
Kurtosis:                      26.900   Cond. No.                     9.15e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.15e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""